In [11]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from itertools import izip
from scipy.stats import mode
from mpl_toolkits.mplot3d import Axes3D
#%matplotlib inline

def calcsCluster(samples, N, decimals = 2, n_clusters = 3):
    #select the parameters with a clustering approach

    #FIX for covariance matricies
    sampled_as = samples[:, :N].reshape((-1))
    sampled_varXs = samples[:,N:2*N].reshape((-1))
    sampled_varYs = samples[:,2*N:3*N].reshape((-1))
    #sampled_covs = samples[:,3*N:].reshape((-1))
    #print sampled_as.shape, sampled_varXs.shape, sampled_varYs.shape
    data = np.c_[sampled_as, sampled_varXs,sampled_varYs]#,sampled_covs]
    
    #reshape the data such that it's 2-D, with amps on one axis and radii on the other.

    #n_clusters = N
    #fit to clusters
    k_means = KMeans(init = 'k-means++',n_clusters = n_clusters, n_init = 50)
    labels = k_means.fit_predict(data)

    from itertools import cycle
    colors = cycle(['r', 'b', 'g', 'm', 'y', 'c'])
    fig = plt.figure()
    ax = fig.add_subplot(111, projection = '3d')
    nSamples = 10000
    data = data[:nSamples]
    labels = labels[:nSamples]
    for cluster, color in izip(xrange(n_clusters), colors):
        '''
        plt.subplot(311)
        plt.title('Amps v. X')
        plt.scatter(data[labels==cluster, 0], data[labels==cluster,1], color = color)
        plt.subplot(312)
        plt.title('Amps v. Y')
        plt.scatter(data[labels==cluster,0], data[labels==cluster,2], color = color)
        plt.subplot(313)
        plt.title('X v. Y')
        plt.scatter(data[labels==cluster,1], data[labels==cluster,2], color = color)
        '''
        ax.scatter(data[labels == cluster, 0],data[labels == cluster, 1],data[labels == cluster, 2], color = color )

    plt.show()

    #round the data for binning and mode selection
    roundData = np.round_(data, decimals = decimals)
    clusters = [roundData[labels == i] for i in xrange(n_clusters)]

    #get a list of the highest points and their average count
    allModes = []
    for cluster in clusters:
           point, counts = mode(cluster)
           #TODO don't forget to add cov here
           allModes.append((point[0][0], point[0][1],point[0][2], counts.mean()))

    allModes.sort(key = lambda x:x[-1], reverse = True) #sort by highest count
    allModes = np.array(allModes)
    #select the 1st N popular points in the clusters
    modes = allModes[:N,:4]

    calc_as, calc_varXs,calc_varYs = modes[:,0], modes[:,1], modes[:,2]#, modes[:,3]
    return calc_as, calc_varXs,calc_varYs#, calc_covs 

In [12]:
samples_file = '/home/mclaughlin6464/Documents/Research/output/CFHTLS_072_2003_samples'
samples = np.loadtxt(samples_file, delimiter = ',')

In [14]:
calc_as, calc_varXs, calc_varYs = calcsCluster(samples, 2, n_clusters = 4)